In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from hit_rate import HitRate
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[*]") \
                    .config('spark.ui.showConsoleProgress', 'false')\
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 12:58:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=MovieRecomender>
Spark App Name : MovieRecomender


In [3]:
import os 
os.chdir ('/home/mt/MovieRecomender')
#change to your working directory
%pwd

'/home/mt/MovieRecomender'

In [4]:
#df_m = pd.read_csv("data/movies.dat", engine='python', sep='::', names=["MovieID", "Title", "Genres"],encoding='ISO-8859-1')
#df_m = spark.createDataFrame(df_m)

df_r = pd.read_csv("data/ratings.dat", engine='python', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])
df_r = spark.createDataFrame(df_r)

##df_u = spark.createDataFrame(df_u)

In [5]:
#df_m.printSchema()
#df_m.show(3)

In [6]:
df_r.printSchema()
df_r.show(3)

root
 |-- UserID: long (nullable = true)
 |-- MovieID: long (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)

22/10/05 12:59:12 WARN TaskSetManager: Stage 0 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
+------+-------+------+---------+
only showing top 3 rows



In [7]:
#df_u.printSchema()
#df_u.show(3)

In [8]:
df=df_r#.join(df_r,"MovieID","inner")#.join(df_u,"UserID","inner")
df.printSchema()
df.show(3)

root
 |-- UserID: long (nullable = true)
 |-- MovieID: long (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)

22/10/05 12:59:13 WARN TaskSetManager: Stage 1 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
+------+-------+------+---------+
only showing top 3 rows



In [9]:
df=df.select(["UserID","MovieID","Rating"])
df.na.drop()
df.show(3)

22/10/05 12:59:13 WARN TaskSetManager: Stage 2 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     1|   1193|     5|
|     1|    661|     3|
|     1|    914|     3|
+------+-------+------+
only showing top 3 rows



In [10]:
(train, test) = df.randomSplit([0.8, 0.2])

In [11]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False,coldStartStrategy="drop")

In [12]:
grid_search = ParamGridBuilder().addGrid(als.rank,[30,50]).addGrid(als.maxIter,[15,20,50]).addGrid(als.regParam, [0.05] ).build()

In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 

In [14]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=evaluator, numFolds=5)

In [15]:
spark.sparkContext.setCheckpointDir('checkpoint/')
cv_fitted=cv.fit(train)

22/10/05 12:59:15 WARN TaskSetManager: Stage 3 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 12:59:16 WARN TaskSetManager: Stage 4 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 12:59:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/05 12:59:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/10/05 12:59:37 WARN TaskSetManager: Stage 93 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 12:59:40 WARN TaskSetManager: Stage 126 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 12:59:40 WARN TaskSetManager: Stage 127 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 12:59:55 WARN TaskSetManager: Stage

In [16]:
print(cv_fitted.bestModel.rank, cv_fitted.bestModel._java_obj.parent().getMaxIter(),cv_fitted.bestModel._java_obj.parent().getRegParam())

50 15 0.05


In [19]:
evaluator.evaluate(cv_fitted.transform(test).na.drop())

22/10/05 13:17:22 WARN TaskSetManager: Stage 6263 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.


0.8546624223994767

In [18]:
hr_evaluator = HitRate(predictionCol='prediction', labelCol='rating', userCol='userId')
model = als.fit(train)
predictions = model.transform(test)
hr_evaluator.evaluate(predictions)

22/10/05 13:12:19 WARN TaskSetManager: Stage 6016 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 13:12:20 WARN TaskSetManager: Stage 6017 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 13:12:24 WARN TaskSetManager: Stage 6078 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-----+
|userId|count|
+------+-----+
|     1|   10|
|     5|   10|
|     6|   10|
|     7|    8|
|     9|   10|
|    10|   10|
|    17|   10|
|    19|   10|
|    22|   10|
|    25|   10|
|    26|   10|
|    27|   10|
|    28|   10|
|    29|   10|
|    31|   10|
|    32|    8|
|    33|   10|
|    34|   10|
|    39|   10|
|    41|    5|
+------+-----+
only showing top 20 rows

22/10/05 13:12:26 WARN TaskSetManager: Stage 6138 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 13:12:28 WARN TaskSetManager:

1.0